In [ ]:
import requests
import shutil
import os
from pathvalidate import sanitize_filename

end = False
i = 1
cwd = os.getcwd()
    

#downloads image from xkcd website using json api. Uses the name, year, and month from the json metadata to organize.

def download(metadata):

    metadata = response.json()

    if(os.path.isfile(cwd + '/xkcd/' + metadata['year'] + '/' + metadata['month'] + '/' + sanitize_filename(metadata['safe_title']) + '.png') == False):

        comic = requests.get(metadata['img'], stream = True)

        if comic.status_code == 200:
            if(os.path.isdir(cwd + '/xkcd/' + metadata['year']) == False):
                os.mkdir(cwd + '/xkcd/' + metadata['year'])
            if(os.path.isdir(cwd + '/xkcd/' + metadata['year'] + '/' + metadata['month'] + '/') == False):
                os.mkdir(cwd + '/xkcd/' + metadata['year']+ '/' + metadata['month'] + '/')
            with open(cwd + '/xkcd/' + metadata['year'] + '/' + metadata['month'] + '/' + sanitize_filename(metadata['safe_title']) + '.png','wb') as f:
                shutil.copyfileobj(comic.raw, f)
            print(str(i) + ': Image sucessfully Downloaded: ',metadata['safe_title'])
        else:
            print('Image Couldn\'t be retrieved')
    else:
        print(str(i) + ': Image Already Downloaded: ',metadata['safe_title'])

#loops until it reaches a comic id that doesn't exist. Skips 404 as it that id is used by the pages 404 error page.

while end == False:       

    if i != 404:
        
        url = "https://xkcd.com/{0}/info.0.json".format(i)

        if(os.path.isdir(cwd + '/xkcd/') == False):
            os.mkdir(cwd + '/xkcd/')

        try:
            response = requests.get(url)
            response.raise_for_status()
            download(response.json())
        except requests.exceptions.HTTPError as errcode:
            print(errcode)
            end = True
    i = i + 1



In [12]:
#Converts File to .cbr archive for easier viewing

if(os.path.isdir(cwd + '/cbr/') == False):
    os.mkdir(cwd + '/cbr/')

shutil.make_archive('temp', 'zip','xkcd')
shutil.copyfile('temp.zip', cwd+'/cbr/xkcd.cbr')
os.remove('temp.zip')